<a href="https://colab.research.google.com/github/chobbong/AI18_ChoYunSeo_Section2/blob/main/N323a_Language_Modeling_with_RNN_AI18_%EC%A1%B0%EC%9C%A4%EC%84%9C_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 3 / SPRINT 2 / Assignment 3*

--- 

# Language Modeling with RNN



## Code

다음 링크는 LSTM을 사용하여 Spam 메시지 분류를 수행한 캐글 노트북입니다. => [Link](https://www.kaggle.com/kredy10/simple-lstm-for-text-classification) <br/>

위 노트북에서 사용한 코드를 참고하여<br/>
캐글 데이터셋인 [Women's E-Commerce Clothing Reviews](https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews) 를 분류해 보세요.

- 분류에 사용될 텍스트 데이터 : **`Review Text`** 열을 사용합니다.
- 레이블(label) 데이터 : **`Recommended IND`** 열을 사용합니다.

### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다.
- 필요없는 열(column)을 삭제합니다.

In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from plotly import graph_objs as go

%matplotlib inline

In [9]:
from google.colab import files

file = files.upload()

Saving Womens Clothing E-Commerce Reviews.csv to Womens Clothing E-Commerce Reviews.csv


In [11]:
df = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [13]:
df.isnull().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [15]:
df1 = df.drop(['Title', 'Clothing ID', 'Positive Feedback Count'], axis=1)
df1.head()

,Unnamed: 0,Age,Review Text,Rating,Recommended IND,Division Name,Department Name,Class Name
0,0,33,Absolutely wonderful - silky and sexy and comf...,4,1,Initmates,Intimate,Intimates
1,1,34,Love this dress! it's sooo pretty. i happene...,5,1,General,Dresses,Dresses
2,2,60,I had such high hopes for this dress and reall...,3,0,General,Dresses,Dresses
3,3,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,General Petite,Bottoms,Pants
4,4,47,This shirt is very flattering to all due to th...,5,1,General,Tops,Blouses


In [16]:
df1.isnull().sum()

Unnamed: 0           0
Age                  0
Review Text        845
Rating               0
Recommended IND      0
Division Name       14
Department Name     14
Class Name          14
dtype: int64

In [23]:
df2 = df1.dropna(subset=['Review Text', 'Division Name', 'Department Name', 'Class Name'], axis=0)
df2 = df2.reset_index(drop=True)

In [24]:
df2.isnull().sum()

Unnamed: 0         0
Age                0
Review Text        0
Rating             0
Recommended IND    0
Division Name      0
Department Name    0
Class Name         0
dtype: int64

In [26]:
df3 = df2.drop(['Unnamed: 0'], axis=1)
df3.head()

,Age,Review Text,Rating,Recommended IND,Division Name,Department Name,Class Name
0,33,Absolutely wonderful - silky and sexy and comf...,4,1,Initmates,Intimate,Intimates
1,34,Love this dress! it's sooo pretty. i happene...,5,1,General,Dresses,Dresses
2,60,I had such high hopes for this dress and reall...,3,0,General,Dresses,Dresses
3,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,General Petite,Bottoms,Pants
4,47,This shirt is very flattering to all due to th...,5,1,General,Tops,Blouses


In [27]:
df3.shape[0]

22628

In [31]:
recommended = (
    df3['Recommended IND']
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={'index':'Recommended', 'Recommended IND':'Count'})
    .sort_values(by=['Recommended'], ascending=True)
    .replace([0, 1], ['No', 'Yes'])   
          )  

colors = ['#f6b220','#0E2F44']

fig = go.Figure(data=[go.Pie(labels=recommended['Recommended'], 
                             values=recommended['Count'])])

fig.update_traces(hoverinfo='percent', 
                  textinfo='label', 
                  textfont_size=20,
                  marker=dict(colors=colors, 
                              line=dict(color='white', width=1)))

fig.update_layout(showlegend=False, 
                  title_text="<b>Recommended</b> Distribution",
                  title_x=0.5,
                  font=dict(family="Rockwell, sans-serif", size=25, color='#000000'))

fig.show()

In [32]:
classes = (
    df3
    .groupby(['Recommended IND', 'Class Name'])
    .size()
    .to_frame()
    .rename(columns={0:'Count'})
    .reset_index()
          )  

# get proportions in each class
a = classes.groupby('Class Name')['Count'].transform('sum')

classes['Count'] = classes['Count'].div(a)

# pivot table
classes = classes.pivot(index='Class Name', columns='Recommended IND')  

fig = go.Figure()
fig.add_trace(go.Bar(
    y=classes.index,
    x=classes.iloc[:,0],
    name='Not Recommended',
    orientation='h',
    marker=dict(
        color='#f6b220')
    ))

fig.add_trace(go.Bar(
    y=classes.index,
    x=classes.iloc[:,1],
    name='Recommended',
    orientation='h',
    marker=dict(
        color='#0E2F44')
    ))

fig.update_layout(barmode='stack')

fig.update_layout(
                title = 'Distribution of <b>Product Class</b> by Recommendation ',
                barmode='stack', 
                autosize=False,
                width=680,
                height=800,
                font=dict(family="Rockwell, sans-serif", size=18, color='#000000'),
                margin=dict(
                  l=150,
                  r=100,
                   b=30,
                   t=100,
                   pad=4
                          ))
fig.layout.xaxis.tickformat = ',.0%'

fig.show()

In [34]:
rating = (
    df3['Rating']
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={'index':'Rating', 'Rating':'Count'})
    .sort_values(by=['Rating'], ascending=True)   
          ) 

rating['percent'] = ((rating['Count'] / rating['Count'].sum())*100).round(2).astype(str) + '%'

colors = ['#0E2F44',] * 5
colors[4] = '#f6b220'

fig = go.Figure(go.Bar(
            y=rating['Count'],
            x=rating['Rating'],
            marker_color=colors,
            text=rating['percent']
                        ))

fig.update_traces(texttemplate='%{text}', 
                  textposition='outside',
                  cliponaxis = False,
                  hovertemplate='<b>Rating</b>: %{x}<br><extra></extra>'+  # <extra></extra> removes trance 0
                                '<b>Count</b>: %{y}',
                  textfont_size=18)
                  
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
 
fig.update_layout(coloraxis=dict(colorscale='Teal'),
                  showlegend=False, 
                  plot_bgcolor='#F7F7F7', 
                  margin=dict(pad=20),
                  paper_bgcolor='#F7F7F7',
                  height=500,
                  yaxis={'showticklabels': False},
                  yaxis_title=None,
                  xaxis_title=None,
                  title_text="<b>Rating</b> Distribution",
                  title_x=0.5,
                  font=dict(family="Rockwell, sans-serif", size=22, color='#000000'),
                  title_font_size=35)


fig.show()

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 20%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words=3000` 으로 설정합니다.
- pad_sequence의 `maxlen=400` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=128, epochs=10, validation_split=0.2` 로 설정합니다.
- EarlyStopping을 적용합니다. 파라미터는 `monitor='val_loss',min_delta=0.0001, patience=3` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [40]:
X = df3['Review Text']
Y = df3['Recommended IND']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

max_words = 3000
max_len = 400
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences, maxlen=max_len)

def create_model():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

model = create_model()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3)

history = model.fit(sequences_matrix, Y_train, batch_size=128, epochs=10, validation_split=0.2, callbacks=[early_stopping])

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen=max_len)

loss, acc = model.evaluate(test_sequences_matrix, Y_test)

print("Loss and accuracy on the test set: ", [loss, acc])

Epoch 1/10
114/114 [==============================] - 94s 799ms/step - loss: 0.3656 - accuracy: 0.8479 - val_loss: 0.3015 - val_accuracy: 0.8672
Epoch 2/10
114/114 [==============================] - 86s 758ms/step - loss: 0.2421 - accuracy: 0.8983 - val_loss: 0.2731 - val_accuracy: 0.8724
Epoch 3/10
114/114 [==============================] - 86s 756ms/step - loss: 0.2187 - accuracy: 0.9113 - val_loss: 0.3031 - val_accuracy: 0.8605
Epoch 4/10
114/114 [==============================] - 86s 751ms/step - loss: 0.1983 - accuracy: 0.9191 - val_loss: 0.2789 - val_accuracy: 0.8763
Epoch 5/10
142/142 [==============================] - 9s 61ms/step - loss: 0.3005 - accuracy: 0.8637
Loss and accuracy on the test set:  [0.3005303740501404, 0.8636765480041504]


In [ ]:
# [0.3005303740501404, 0.8636765480041504]